# Semantic Similarity Example

## Installations

In [1]:
!pip install --force-reinstall prompttools

## Setup imports and API keys

First, we'll need to set our API keys. If we are in DEBUG mode, we don't need to use real OpenAI or Hegel AI API keys, so for now we'll set them to empty strings.

In [2]:
import os
os.environ['DEBUG'] = ""  # Set to "1" if you want to use debug mode.
os.environ['HEGELAI_API_KEY'] = ""  # Optional, it will be needed to use with `HegelScribe` to persist/visualize your experiments
os.environ['OPENAI_API_KEY'] = ""

Then we'll import the relevant `prompttools` modules to setup our experiment.

In [3]:
from typing import Dict, Tuple
from prompttools.harness import PromptTemplateExperimentationHarness

## Run experiments

Next, we create our test inputs. For this example, we'll use a prompt template, which uses [jinja](https://jinja.palletsprojects.com/en/3.1.x/) for templating.

In [3]:
prompt_templates = ["Answer the following question: {{input}}", "Respond the following query: {{input}}"]
user_inputs = [{"input": "Who was the first president?"}, {"input": "Who was the first president of India?"}]

Now we can define an experimentation harness for our inputs and model. We could also pass model arguments if, for example, we wanted to change the model temperature.

In [4]:
harness = PromptTemplateExperimentationHarness("text-davinci-003", 
                                               prompt_templates, 
                                               user_inputs,
                                               model_arguments={"temperature": 0.9})

We can then run the experiment to get results.

In [5]:
harness.run()
harness.visualize()

messages   
0  Answer the following question: Who was the first president?           \
1  Answer the following question: Who was the first president of India?   
2  Respond the following query: Who was the first president?              
3  Respond the following query: Who was the first president of India?     

                                                  response(s)   latency  
0  [\n\nThe Los Angeles Dodgers won the World Series in 2020]  0.000005  
1  [\n\nThe Los Angeles Dodgers won the World Series in 2020]  0.000002  
2  [\n\nThe Los Angeles Dodgers won the World Series in 2020]  0.000002  
3  [\n\nThe Los Angeles Dodgers won the World Series in 2020]  0.000002

You can use the `pivot` keyword argument to view results by the template and inputs that created them.

In [6]:
harness.visualize(pivot=True)

prompt_template                                                       Answer the following question: {{input}}   
user_input                                                                                                       
{'input': 'Who was the first president of India?'}  [\n\nThe Los Angeles Dodgers won the World Series in 2020]  \
{'input': 'Who was the first president?'}           [\n\nThe Los Angeles Dodgers won the World Series in 2020]   

prompt_template                                                         Respond the following query: {{input}}  
user_input                                                                                                      
{'input': 'Who was the first president of India?'}  [\n\nThe Los Angeles Dodgers won the World Series in 2020]  
{'input': 'Who was the first president?'}           [\n\nThe Los Angeles Dodgers won the World Series in 2020]

## Evaluate the model response

To evaluate the results, we'll define an eval function. We can use semantic distance to check if the model's response is similar to our expected output.

In [10]:
from prompttools.utils import similarity


EXPECTED = {"Who was the first president?": "George Washington",
            "Who was the first president of India?": "Rajendra Prasad"}

def extract_responses(output) -> str:
    return [choice["text"] for choice in output["choices"]]


def measure_similarity(
    input_pair: Tuple[str, Dict[str, str]], results: Dict, metadata: Dict
) -> float:
    """
    A simple test that checks semantic similarity between the user input
    and the model's text responses.
    """
    distances = [
        similarity.compute(EXPECTED[input_pair[1]["input"]], response)
        for response in extract_responses(results)
    ]
    return min(distances)
    


Finally, we can evaluate and visualize the results.

In [9]:
harness.evaluate("similar_to_expected", measure_similarity)
harness.visualize()

[2023-07-05 15:08:51,242] WARNING in Collection: No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
[2023-07-05 15:08:51,469] WARNING in Collection: No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
[2023-07-05 15:08:51,723] WARNING in Collection: No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
[2023-07-05 15:08:51,968] WARNING in Collection: No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


messages   
0  Answer the following question: Who was the first president?           \
1  Answer the following question: Who was the first president of India?   
2  Respond the following query: Who was the first president?              
3  Respond the following query: Who was the first president of India?     

                                                  response(s)   latency   
0  [\n\nThe Los Angeles Dodgers won the World Series in 2020]  0.000005  \
1  [\n\nThe Los Angeles Dodgers won the World Series in 2020]  0.000002   
2  [\n\nThe Los Angeles Dodgers won the World Series in 2020]  0.000002   
3  [\n\nThe Los Angeles Dodgers won the World Series in 2020]  0.000002   

   similar_to_expected  
0  0.965657             
1  1.021784             
2  0.965657             
3  1.021784